In [1]:
#Import libraries
import matplotlib.pyplot as plt
import hierarchy_library as hili

from qiskit import quantum_info
import numpy as np
from scipy.linalg import sqrtm
import pickle
from multiprocessing import Pool
import time





/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 2.0.5-build-libtorrent-rasterbar-qrM5vM-libtorrent-rasterbar-2.0.5-bindings-python is an invalid version and will not be supported in a future release
  warnings.warn(


In [ ]:
#generate random states
N = 10000000
p = Pool(24)

globalseed = np.random.randint(97956999)
print(globalseed)

results = p.map_async(hili.generate_and_calculate_state, range(globalseed+547,globalseed+N+547), chunksize=1)
while True:
    completed = N - results._number_left
    print(f"Generating state {completed} of {N} ({(completed/N*100):.2f}%)",end="\r")
    if results.ready(): break
    time.sleep(1)
    
results = results.get()

83743879


In [ ]:
random_states,labels_true,purities,collectives,R,measures = zip(*results)

#inspect generated states
#plt.hist(labels_true,5)
#plt.hist(purities)


#save generated states
data = {
    #"random_states" : random_states,
    "labels_true" : np.array(labels_true),
    #"purities" : purities,s
    "collectives" : np.array(collectives),
    "R": np.array(R)
}

with open("random_states10Mt.pickle","wb") as f:
    pickle.dump(data,f)

In [ ]:
##### CHECKING MATH ################
p = 0.33
Werner = p*hili.Bell + (1-p)*np.eye(4)/4
rho = quantum_info.DensityMatrix(Werner)

#rho = generate_random_state(123)
#rho = quantum_info.DensityMatrix(np.array([[1,1,0,0], [1,1,0,0], [0,0,0,0], [0,0,0,0]])/2)

print(hili.classify_analytically_hierarchy(rho))
R = hili.classify_analytically_hierarchy(rho)[1]
cv = hili.calculate_collective(rho)

cm = np.array([[cv[0], cv[1], cv[2], cv[3]], [cv[1], cv[4], cv[5], cv[6]], [cv[2], cv[5], cv[7], cv[8]], [cv[3], cv[6], cv[8], cv[9]]])

tmp = np.tensordot(hili.MinBase2PauliMatrix, cm, (1,1))
RfromMinBasis = np.tensordot(tmp, hili.MinBase2PauliMatrix, (1,1))


print(RfromMinBasis)
print(R)


In [ ]:
with open("random_states1Mc.pickle","rb") as f:
    dataA = pickle.load(f)

with open("random_states10M_equipartite.pickle","rb") as f:
    dataB = pickle.load(f)

#from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(dataA["labels_true"],dataB["labels_true"])

#print(cm)


found = 0
for f in dataA["collectives"]:
    if f in dataB["collectives"]: found +=1

print(found)